# Math Forward Neuron Network
A simple forward network for computing binary math operations. Where two Neurons are the decimal input, and the third is the math operation.

In [ ]:
%pip install -q imageio pydot tensorflow-gpu==2.9.3 keras matplotlib graphviz moviepy

In [2]:
import numpy
import os.path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LeakyReLU
from keras.layers import Flatten
from tensorflow import keras
from keras.layers import Dropout
from keras.layers import LeakyReLU, ReLU
from random import Random
import keras_tuner as kt

/media/programming/projects/machineLearning/ML/venv/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [3]:
tf.__version__

'2.9.3'

### Configure Output Directory
Set the output directory where all the results will be storaged.

In [4]:
root_dir = "result/forward_math_neuron network"
if not os.path.exists(root_dir):
    os.mkdir(root_dir)


## Import all Modules and Configure GPU

In [5]:
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" # Debug only
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # mixed_precision.set_global_policy('mixed_float16')
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Initialize tf.distribute.MirroredStrategy
strategy = tf.distribute.MirroredStrategy(devices=None)
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


2023-05-21 10:31:06.367573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-21 10:31:06.390889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-21 10:31:06.391083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-21 10:31:06.392278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Generate DataSet

In [6]:
def generate_number_set(size):
	number_Y = []
	number_X = []
	random = Random()
	for i in range(0, size):
		a = random.random() * 100
		b = random.random() * 100
		math_operator = float(random.randint(0, 2))

		number_X.append([a, b, math_operator])

		if math_operator == 0:
			number_Y.append(a + b)
		elif math_operator == 1:
			number_Y.append(a - b)
		elif math_operator == 2:
			number_Y.append(a * b)
#        elif math_operator == 3:
#            number_Y.append(a / b)

	return numpy.asarray(number_X), np.asarray(number_Y)


## Model Generator with Hyperparameter

In [7]:

def make_generator_model(hp, input, output):
	model = Sequential()

	kernel_init = hp.Choice('kernel_initializer', ['uniform', 'lecun_uniform', 'normal', 'zero',
												   'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])

	hp0_units = hp.Int('dense0_units', min_value=32, max_value=512, step=32)
	hp1_units = hp.Int('dense1_units', min_value=32, max_value=512, step=32)
	hp2_units = hp.Int('dense2_units', min_value=32, max_value=512, step=32)

	model.add(Dense(hp0_units, input_dim=input, kernel_initializer=kernel_init, dtype='float32'))
	model.add(ReLU())
	model.add(Dense(hp1_units, dtype='float32', kernel_initializer=kernel_init))
	model.add(ReLU())
	model.add(Dense(hp2_units, dtype='float32', kernel_initializer=kernel_init))
	model.add(ReLU())
	model.add(Dense(output, dtype='float32', kernel_initializer=kernel_init))

	hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-1])

	hp_optimizer = hp.Choice('optimizer', ['sgd', 'adam', 'rmsprop'])

	optimizer = tf.keras.optimizers.get(hp_optimizer)
	optimizer.learning_rate = hp_learning_rate

	def ssim_loss(y_true, y_pred):
		return (tf.reduce_mean(y_true - y_pred , axis=-1))

	model.compile(optimizer=optimizer,
					loss='mae',
					metrics=['accuracy'])

	return model


In [8]:
input = 3
output = 1
EPOCH = 64
BATCH_SIZE = 4096
train_X, train_y = generate_number_set(5120000)

def model_builder(hp):
    model = make_generator_model(hp, input, output)
    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=48,
                     factor=4,
                     directory=os.path.join(root_dir, 'cache'),
                     project_name=str.format('Math'))

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_X, train_y, epochs=50, batch_size=BATCH_SIZE, validation_split=0.25, callbacks=[stop_early, tf.keras.callbacks.TerminateOnNaN()])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

best_models = tuner.get_best_models(8)

train_X, train_y = generate_number_set(5120000)

cnn_model = tuner.hypermodel.build(best_hps)
cnn_model.summary()
cnn_model_history = cnn_model.fit(train_X, train_y, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=0.2)


Trial 44 Complete [00h 00m 08s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 07m 42s
INFO:tensorflow:Oracle triggered exit
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 96)                384       
                                                                 
 re_lu_3 (ReLU)              (None, 96)                0         
                                                                 
 dense_5 (Dense)             (None, 128)               12416     
                                                                 
 re_lu_4 (ReLU)              (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 480)               61920     
                                                                 
 re_lu_5 (ReLU)              (None,